In [1]:
import pandas as pd
import nltk
import os

In [2]:
os.listdir()

['.gitignore',
 '.ipynb_checkpoints',
 '01. DMZ 피드백 재시도.ipynb',
 '01. DMZ 피드백 재시도_2.ipynb',
 'UseData',
 '네트워크_dmz_feedback.ipynb']

In [3]:
url = 'UseData/'
os.listdir(url)

['(학술자료_해외) animal.xlsx',
 '(학술자료_해외) Biodiversity.xlsx',
 '(학술자료_해외) plant.xlsx',
 '(학술자료_해외) preservation or conservation.xlsx',
 '(학술자료_해외) south korea.xlsx']

In [4]:
files = pd.DataFrame({
    'FileName' : os.listdir(url),
    'Lang' : ['en', 'en', 'en', 'en', 'en']
})
files

,FileName,Lang
0,(학술자료_해외) animal.xlsx,en
1,(학술자료_해외) Biodiversity.xlsx,en
2,(학술자료_해외) plant.xlsx,en
3,(학술자료_해외) preservation or conservation.xlsx,en
4,(학술자료_해외) south korea.xlsx,en


In [15]:
# 함수 정의
# 파일을 읽고, 어간 별 분리를 한 후 갯수를 count, gephi에 넣을 수 있게 가공.
def Processor(url, filename, lang):
    df = pd.read_excel(url + filename, sheet_name = 'Sheet1', usecols=['Contents'])
    
    resultList = []
    wordDict = {}
    
    # 숫자 제거, 길이가 3 이하인 단어만 제거 및 lower 전처리
    df['Contents'] = df['Contents'].str.replace('[^a-zA-Z]+', ' ')
    df['Contents'] = df['Contents'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
    df['Contents'] = df['Contents'].apply(lambda x: x.lower())
    
    # 전처리된 문장들에 대해 Loop
    # 띄어쓰기 기준으로 토큰화 및 태깅작업 진행
    for sentence in df['Contents']:
        words = nltk.tokenize.word_tokenize(sentence) # 띄어쓰기 기준 단어 토큰화
        word_with_tag = nltk.tag.pos_tag(words) # 단어들에 대한 태깅 작업
        wordList = [word for word, tag in word_with_tag if ('NN' in tag)] # 명사 만 추출 
        
        # 추출 된 단어 Counting
        for word in wordList:
            if word in wordDict.keys():
                wordDict[word] += 1
            else:
                wordDict[word] = 1
        resultList.append(wordList)
        # print(resultList[:2]) # 확인용
    
    # 상위 100개 단어만 추출
    stop = [x[0] for x in sorted(wordDict.items(), key = (lambda x: x[1]), reverse = True)[:100]]
    
    # Gephi에서 받을 수 있는 형태로 저장
    results = []
    for r in resultList:
        imsi = []
        for _r in r:
            if _r in stop:
                imsi.append(_r)
        if len(imsi) != 0 :
            results.append(imsi)
    pd.DataFrame(results).to_csv('./Outputs/' + file[:-5] + '.csv', index = False, header = False)

In [16]:
# 파일 목록(DataFrame)에 있는 파일에 대해 작업 수행
for index in range(len(files)):
    file = files['FileName'][index]
    lang = files['Lang'][index]
    Processor(url, file, lang)

C:\Users\skdbs\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


    1. Gephi 형태로 저장 --> ?
    일단 Gephi 넣어보자.

    그래프는 생성 됨.
    Gephi Preview가 근데 안나와서 재설치 해야 할 듯.